In [ ]:
import os
import sys
import time

sys.path.append('../..')
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt
import mdtraj as md
import numpy as np
import tensorflow as tf

from Constants import EV_TO_KJ, H_TO_KJ, BOHR_TO_ANGSTROM

from ase import Atoms
from ase.io import read
from ase.io.trajectory import Trajectory
from ase.thermochemistry import HarmonicThermo, CrystalThermo

from scipy.constants import Boltzmann, Avogadro
from scipy.stats import linregress
from scipy.integrate import romb, simpson
from ase.geometry.cell import cell_to_cellpar

import mdtraj.utils.unit.unit_definitions as u

In [ ]:
SYSTEMS_DATA = np.load('../../../data/test_sets/DATA_CSP_ranking.npy', allow_pickle=True).item()


RT = lambda T: (Boltzmann * Avogadro) * T / 1000

RMSE = lambda x, y: np.round(np.sqrt(np.mean(np.square(np.subtract(x, y)))), 1)
MAE = lambda x, y: np.round(np.mean(np.abs(np.subtract(x, y))), 1) 
ME = lambda x, y: np.round(np.mean(np.subtract(x, y)), 1)
STD = lambda x, y: np.round(np.std(np.subtract(x, y)), 1)
MAE = lambda x, y: np.round(np.mean(np.abs(np.subtract(x, y))), 1)

In [ ]:
SYSTEM_NAME = 'XXIII'
FOLDER_PATH = 'data'

if SYSTEM_NAME == 'XXII':
    T = 150
else:
    T = 300
N_REPLICA = 1
T_offset = 200

MOL_SIZE = len(SYSTEMS_DATA[SYSTEM_NAME]['N100']['monomer_symbols'])
RESULTS = np.load(f'data/RESULTS_{SYSTEM_NAME}.npy', allow_pickle=True).item()
MIN_CELLS = np.load(f'systems/MIN_CELLS_{SYSTEM_NAME}_IND0.npy', allow_pickle=True).item()
MIN_STRUCTURES = np.load(f'data/STRUCTURES_{SYSTEM_NAME}.npy', allow_pickle=True).item()
MD_TERMS = np.load(f'data/MD_TERMS_{SYSTEM_NAME}.npy', allow_pickle=True).item()

In [ ]:
DATA = {}

In [ ]:
names, ana2b_E0, ana2b_ET, ana2b_dGh, ana2b_dAh, ana2b_gibbs_term, pbe_mbd, pbe0_mbd, pbe_ts, experimental = [], [], [], [], [], [], [], [], [], []
Ns = []
for poly_key in RESULTS[SYSTEM_NAME]:
    if poly_key in MD_TERMS:
        try:
            N = len(SYSTEMS_DATA[SYSTEM_NAME][poly_key]['uc_symbols']) // len(SYSTEMS_DATA[SYSTEM_NAME][poly_key]['monomer_symbols']) 
            volume = np.linalg.det(MIN_CELLS[poly_key][0]) / N
            E0 = RESULTS[SYSTEM_NAME][poly_key]['total']
            phonon_results = np.load(f'vibdata_phonon/{SYSTEM_NAME}/phonons_{poly_key}.npy', allow_pickle=True).item()
            thermo = CrystalThermo(phonon_energies=phonon_results['phonon_energies'],
                               phonon_DOS=phonon_results['phonon_DOS'],
                               potentialenergy=0,
                               formula_units=N)
            ediffs = np.load(os.path.join('data_ah/', f'{SYSTEM_NAME}/{SYSTEM_NAME}_{poly_key}_ediffs.npy'), allow_pickle=True).item()
            lambdas = [key for key in ediffs]
            ediffs = [ediffs[lamb] for lamb in lambdas]
            dA = (np.trapz(ediffs, x=lambdas) * EV_TO_KJ) / N
            plt.scatter(lambdas, ediffs)
            plt.plot(lambdas, ediffs)
            Ns.append(N)
            dG = thermo.get_helmholtz_energy(T) * EV_TO_KJ
            ana2b_E0.append(E0)
            ana2b_dGh.append(dG)
            ana2b_dAh.append(dA)
            ana2b_ET.append(MD_TERMS[poly_key]['V_pot_mu'])
            ana2b_gibbs_term.append(MD_TERMS[poly_key]['gibbs_term'])
            pbe_ts.append(SYSTEMS_DATA[SYSTEM_NAME][poly_key]['PBE+TS'])
            pbe_mbd.append(SYSTEMS_DATA[SYSTEM_NAME][poly_key]['PBE+MBD'])    
            pbe0_mbd.append(SYSTEMS_DATA[SYSTEM_NAME][poly_key]['PBE0+MBD'])
            experimental.append(SYSTEMS_DATA[SYSTEM_NAME][poly_key]['exp'])
            names.append(poly_key)
        except:
            print(poly_key)
            continue

In [ ]:
[name for name in RESULTS[SYSTEM_NAME] if name not in names]

In [ ]:
# ['N100', 'N90'] # XXII
# ['N43', 'N44', 'N70', 'N39']: XXIII
# ['N59', 'N20', 'N35']: XXVI

In [ ]:
names = np.array(names)
ana2b_E0 = np.array(ana2b_E0)
ana2b_dGh = np.array(ana2b_dGh)
ana2b_ET = np.array(ana2b_ET)
ana2b_dAh = np.array(ana2b_dAh)
ana2b_gibbs_term = np.array(ana2b_gibbs_term)
pbe_ts = np.array(pbe_ts)
pbe_mbd = np.array(pbe_mbd)
pbe0_mbd = np.array(pbe0_mbd)
ana2b_Fh = ana2b_E0 + ana2b_dGh
ana2b_Fa = ana2b_E0 + ana2b_dGh + ana2b_dAh
ana2b_Ga = ana2b_E0 + ana2b_dGh + ana2b_dAh + ana2b_gibbs_term
pbe0_mbd_Gh = [x + y for x, y in zip(pbe0_mbd, (ana2b_dGh + ana2b_dAh + ana2b_gibbs_term)) if x is not None]

ana2b_E0 -= np.amin(ana2b_E0)
ana2b_Fh -= np.amin(ana2b_Fh)
ana2b_Fa -= np.amin(ana2b_Fa)
ana2b_Ga -= np.amin(ana2b_Ga)
ana2b_ET -= np.amin(ana2b_ET)
pbe0_mbd_Gh -= np.amin(pbe0_mbd_Gh)
pbe0_mbd_Gh = np.pad(pbe0_mbd_Gh, [0, len(pbe0_mbd) - len(pbe0_mbd_Gh)], constant_values=None)

experimental = np.array(experimental)

LINEWIDTH = 0.75
LINEWIDTH_EXP = 2

COLOR = '#383838'
COLOR_EXP = '#B8170C'

rankings = [ana2b_E0, ana2b_Fh, ana2b_Fa, ana2b_Ga, ana2b_ET, pbe_ts, pbe_mbd, pbe0_mbd] 
methods = [f'ANA2B$^\infty$ E({0}K)', f'ANA2B$^\infty$ F$_H$({T}K)', 
           f'ANA2B$^\infty$ F$_A$({T}K)', f'ANA2B$^\infty$ G$_A$({T}K)', 
           f'ANA2B$^\infty$ E({T}K)', f'PBE+TS E({0}K)', f'PBE+MBD E({0}K)', f'PBE0+MBD E({0}K)']
offsets = []
if SYSTEM_NAME == 'XXVI':
    for x in range(0, len(rankings)-3):
        if x == len(rankings) - 3:
            i_ = min(np.where(np.isnan(rankings[-1]))[0])
            offsets.append(rankings[x+1][:i_] - rankings[x][:i_])
        else:
            offsets.append(rankings[x+1] - rankings[x])
else:
    for x in range(0, len(rankings)-1):
        offsets.append(rankings[x+1] - rankings[x])


In [ ]:
plt.figure(0, figsize=(8, 4), dpi=500)
for idr, ranking in enumerate(rankings):
    for name, total, exp in zip(names, ranking, experimental):
        if exp:
            plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color=COLOR_EXP, linewidth=LINEWIDTH_EXP, zorder=100)
        else:
            plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color=COLOR, linewidth=LINEWIDTH)

for ido, offset in enumerate(offsets):
    if ido in [0, 1, 2, 3, 5, 6, 7, 8]:
        for E, dE in zip(rankings[ido], offset):
            plt.plot([ido+1 - 0.1, ido + 1 + 0.1], [E, E+dE], color=COLOR, linewidth=0.1)

ax = plt.gca()
ax.set_xlabel('Method')
ax.set_ylabel('Relative Stability [kJ/mol]', labelpad=20)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False) 
plt.xticks(np.arange(0.5, len(methods)), methods, rotation=45)
ax.set_ylim(-0.5)
plt.savefig(f'figures/ranking_{SYSTEM_NAME}.pdf', bbox_inches='tight')

In [ ]:
plt.figure(0, figsize=(8, 4), dpi=500)
for idr, ranking in enumerate(rankings):
    for name, total, exp in zip(names, ranking, experimental):
        if name in ['N85', 'N4', 'N6', 'N39', 'E']:
            if idr == 0:
                if name == 'E':
                    plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color='lightgreen', linewidth=LINEWIDTH_EXP, label='E', zorder=100)
                if name == 'N6':
                    plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color='darkorange', linewidth=LINEWIDTH_EXP, label='C', zorder=100)
                if name == 'N4':
                    plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color='darkblue', linewidth=LINEWIDTH_EXP, label='B', zorder=100)
                if name == 'N85':
                    plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color='red', linewidth=LINEWIDTH_EXP, label='A', zorder=100)
                if name == 'N39':
                    plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color='darkgreen', linewidth=LINEWIDTH_EXP, label='D', zorder=100)
            else:
                if name == 'N85':
                    plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color='red', linewidth=LINEWIDTH_EXP, zorder=100)
                if name == 'N4':
                    plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color='darkblue', linewidth=LINEWIDTH_EXP, zorder=100)
                if name == 'N6':
                    plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color='darkorange', linewidth=LINEWIDTH_EXP, zorder=100)
                if name == 'N39':
                    plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color='darkgreen', linewidth=LINEWIDTH_EXP, zorder=100)
                if name == 'E':
                    plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color='lightgreen', linewidth=LINEWIDTH_EXP, zorder=100)  
        else:
            plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color=COLOR, linewidth=LINEWIDTH)
for ido, offset in enumerate(offsets):
    if ido in [0, 1, 2, 3, 5, 6, 7, 8]:
        for E, dE in zip(rankings[ido], offset):
            plt.plot([ido+1 - 0.1, ido + 1 + 0.1], [E, E+dE], color=COLOR, linewidth=0.1)

ax = plt.gca()
ax.set_xlabel('Method')
ax.set_ylabel('Relative Stability [kJ/mol]', labelpad=20)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False) 
plt.xticks(np.arange(0.5, len(methods)), methods, rotation=45)
ax.set_ylim(-0.5)
#plt.title(SYSTEM_NAME)
plt.legend(frameon=False)
plt.savefig(f'figures/ranking_{SYSTEM_NAME}.pdf', bbox_inches='tight')

In [ ]:
plt.figure(0, figsize=(8, 4), dpi=500)
for idr, ranking in enumerate(rankings):
    for name, total, exp in zip(names, ranking, experimental):
        if exp:
            plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color=COLOR_EXP, linewidth=LINEWIDTH_EXP, zorder=100)
        else:
            plt.plot([idr + 0.1, (idr + 1) - 0.1], [total, total], color=COLOR, linewidth=LINEWIDTH)
for ido, offset in enumerate(offsets):
    if ido in [0, 1, 2, 3, 5, 6, 7, 8]:
        for E, dE in zip(rankings[ido], offset):
            plt.plot([ido+1 - 0.1, ido + 1 + 0.1], [E, E+dE], color=COLOR, alpha=0.5, linewidth=0.1)

ax = plt.gca()
ax.set_xlabel('Method')
ax.set_ylabel('Relative Stability [kJ/mol]', labelpad=20)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False) 
plt.xticks(np.arange(0.5, len(methods)), methods, rotation=60)
ax.set_ylim(-0.5)
plt.savefig(f'figures/ranking_{SYSTEM_NAME}.pdf', bbox_inches='tight')

In [ ]:
Atoms(symbols=SYSTEMS_DATA['XXII']['N2']['monomer_symbols'], positions=SYSTEMS_DATA['XXII']['N2']['monomer_coordinates']).write('figures/XXII.xyz')

In [ ]:
Atoms(symbols=SYSTEMS_DATA['XXVI']['N1']['monomer_symbols'], positions=SYSTEMS_DATA['XXVI']['N1']['monomer_coordinates']).write('figures/XXVI.xyz')

In [ ]:
Atoms(symbols=SYSTEMS_DATA['XXIII']['N85']['monomer_symbols'], positions=SYSTEMS_DATA['XXIII']['N85']['monomer_coordinates']).write('figures/XXIII.xyz')